In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from darts import TimeSeries
from darts.metrics import mape


In [2]:
DATA_ROOT = Path('../../data/')
INPUT_DS_PATH = DATA_ROOT / 'processed' / 'dataset.parquet'
RES_DS_PATH = DATA_ROOT / 'processed' / 'preds.xls'

In [3]:
df_ds = pd.read_parquet(INPUT_DS_PATH)
df_ds.head(3)

,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф,ds,feat_КТ с КУ 1 зона_lag-4_КТ,feat_КТ_lag-1_КТ с КУ 1 зона,feat_МРТ_lag-6_КТ с КУ 2 и более зон,feat_МРТ с КУ 1 зона_lag-1_МРТ,feat_МРТ_lag-1_МРТ с КУ 1 зона,feat_КТ с КУ 1 зона_lag-1_ММГ
1,17.0,6146,43.0,100.0,483,415,169.0,2.0,12450,392.0,2022-01-03,6146.0,43.0,100.0,415.0,169.0,483.0
2,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0,2022-01-10,6146.0,43.0,100.0,415.0,169.0,483.0
3,910.0,12266,430.0,490.0,8791,2162,710.0,14.0,47364,20496.0,2022-01-17,6146.0,424.0,100.0,2156.0,669.0,9567.0


In [4]:
value_cols=[
    'Денситометр',
    'КТ',
    'КТ с КУ 1 зона',
    'КТ с КУ 2 и более зон',
    'ММГ',
    'МРТ',
    'МРТ с КУ 1 зона',
    'МРТ с КУ 2 и более зон',
    'РГ',
    'Флюорограф'
]


In [5]:
# remove 1-st week of the year
df_ds.loc[df_ds['ds'].dt.isocalendar().week == 1, value_cols] = df_ds.shift(-1)

In [6]:
ts_dset = TimeSeries.from_dataframe(df_ds, time_col='ds', value_cols=value_cols, fill_missing_dates=True)
ts_dset

<TimeSeries (DataArray) (ds: 108, component: 10, sample: 1)> Size: 9kB
array([[[1.0260e+03],
        [1.0868e+04],
        [4.2400e+02],
        ...,
        [9.0000e+00],
        [4.8904e+04],
        [2.2626e+04]],

       [[1.0260e+03],
        [1.0868e+04],
        [4.2400e+02],
        ...,
        [9.0000e+00],
        [4.8904e+04],
        [2.2626e+04]],

       [[9.1000e+02],
        [1.2266e+04],
        [4.3000e+02],
        ...,
...
        ...,
        [1.2000e+01],
        [8.0644e+04],
        [5.3940e+03]],

       [[1.8160e+03],
        [3.9390e+03],
        [5.6300e+02],
        ...,
        [1.7000e+01],
        [9.8705e+04],
        [6.5800e+03]],

       [[2.1020e+03],
        [4.5330e+03],
        [5.3300e+02],
        ...,
        [1.7000e+01],
        [9.9858e+04],
        [7.5910e+03]]])
Coordinates:
  * ds         (ds) datetime64[ns] 864B 2022-01-03 2022-01-10 ... 2024-01-22
  * component  (component) object 80B 'Денситометр' 'КТ' ... 'РГ' 'Флюорограф'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [7]:
from sklearn.metrics import mean_absolute_percentage_error
def calc_mape_for_each_column_and_get_mean(val, pred):
    mape = {}
    for column in pred.columns:
        
        print(column, " mape:", round(mape[column],3))

    print("mean mape: ", round(sum(mape.values()) / len(mape),3))
    #return mape


In [22]:
from darts.models.forecasting.baselines import NaiveSeasonal


model = NaiveSeasonal(K=52)
# model = StatsForecastAutoTheta(season_length=52)
# start=0.76 - gives 24 predictions
historical_fcast = model.historical_forecasts(ts_dset, start=0.8, forecast_horizon=4, verbose=True)
print("MAPE = {:.2f}%".format(mape(historical_fcast, ts_dset)))

  0%|          | 0/20 [00:00<?, ?it/s]

MAPE = 54.01%


In [9]:
for c in value_cols:
    print(f"MAPE({c}) = {mape(historical_fcast[c], ts_dset[c]):.2f}")

MAPE(Денситометр) = 36.40
MAPE(КТ) = 12.51
MAPE(КТ с КУ 1 зона) = 17.86
MAPE(КТ с КУ 2 и более зон) = 33.57
MAPE(ММГ) = 33.39
MAPE(МРТ) = 10.45
MAPE(МРТ с КУ 1 зона) = 12.69
MAPE(МРТ с КУ 2 и более зон) = 303.62
MAPE(РГ) = 41.71
MAPE(Флюорограф) = 37.95


In [10]:
historical_fcast[:1]

<TimeSeries (DataArray) (time: 1, component: 10, sample: 1)> Size: 80B
array([[[1.0200e+03],
        [4.6710e+03],
        [4.7800e+02],
        [4.7700e+02],
        [1.2730e+04],
        [1.9130e+03],
        [7.3800e+02],
        [3.0000e+00],
        [4.8811e+04],
        [1.8390e+04]]])
Coordinates:
  * time       (time) datetime64[ns] 8B 2023-09-11
  * component  (component) object 80B 'Денситометр' 'КТ' ... 'РГ' 'Флюорограф'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [11]:
from sklearn.metrics import mean_absolute_percentage_error
for c in value_cols:
    pred = historical_fcast[c].values()
    true = ts_dset[c].values()[-pred.shape[0]:,:]
    m = ~np.isnan(true).any(axis=1)
    pred = pred[m]
    true = true[m]
    #print(pred, true)
    _mape = mean_absolute_percentage_error(pred, true)
    print(f"MAPE({c}) = {_mape:.2f}")

MAPE(Денситометр) = 0.36
MAPE(КТ) = 0.13
MAPE(КТ с КУ 1 зона) = 0.18
MAPE(КТ с КУ 2 и более зон) = 0.34
MAPE(ММГ) = 0.33
MAPE(МРТ) = 0.10
MAPE(МРТ с КУ 1 зона) = 0.13
MAPE(МРТ с КУ 2 и более зон) = 3.04
MAPE(РГ) = 0.42
MAPE(Флюорограф) = 0.38


In [12]:
from darts.models import (StatsForecastAutoARIMA, StatsForecastAutoETS, 
                          StatsForecastAutoTheta, StatsForecastAutoCES,
                          FourTheta, KalmanForecaster, CatBoostModel, Croston,
                          Prophet
                         )
from darts.models.forecasting.theta import SeasonalityMode, ModelMode, TrendMode

from itertools import product

model_grid = {
    NaiveSeasonal: dict(K=[52]),
    #FourTheta: dict(seasonality_period=[51, 50]),  # TDODO investigate parameters
    #Prophet: dict(yearly_seasonality=[True], country_holidays =["RU"]),
    StatsForecastAutoTheta: dict(season_length=[52]), 
    StatsForecastAutoARIMA: dict(season_length=[52]), 
    StatsForecastAutoETS: dict(season_length=[52]), 
    StatsForecastAutoCES: dict(season_length=[52]),
    KalmanForecaster: dict(dim_x=[1]), 
    # CatBoostModel: dict(), # TDODO investigate parameters
    Croston: dict(), # TDODO investigate parameters
}

def create_model(cls, args):
    name = f"{cls.__name__}__{str(args)}"
    print(f"Model name:{name}")
    return {name: cls(**args)}
models = {}
for cls, args in model_grid.items():
    if len(args) == 0:
        models.update(create_model(cls, dict()))
    else:
        for arg in product(*(args.values())):
            p = create_model(cls, dict( zip(args.keys(),arg) ))
            #print(p)
            models.update(p)
                       
models

Model name:NaiveSeasonal__{'K': 52}
Model name:StatsForecastAutoTheta__{'season_length': 52}
Model name:StatsForecastAutoARIMA__{'season_length': 52}
Model name:StatsForecastAutoETS__{'season_length': 52}
Model name:StatsForecastAutoCES__{'season_length': 52}
Model name:KalmanForecaster__{'dim_x': 1}
Model name:Croston__{}


{"NaiveSeasonal__{'K': 52}": NaiveSeasonal(K=52),
 "StatsForecastAutoTheta__{'season_length': 52}": StatsForecastAutoTheta(season_length=52),
 "StatsForecastAutoARIMA__{'season_length': 52}": StatsForecastAutoARIMA(add_encoders=None, season_length=52),
 "StatsForecastAutoETS__{'season_length': 52}": StatsForecastAutoETS(add_encoders=None, season_length=52),
 "StatsForecastAutoCES__{'season_length': 52}": StatsForecastAutoCES(season_length=52),
 "KalmanForecaster__{'dim_x': 1}": KalmanForecaster(dim_x=1, kf=None, add_encoders=None),
 'Croston__{}': Croston(version=classic, alpha_d=None, alpha_p=None, add_encoders=None)}

In [13]:
# make GT
from numpy.lib.stride_tricks import sliding_window_view

gt_vals = []
cols = ['ds'] + value_cols

dd = df_ds[cols].values
s = dd.shape
data = sliding_window_view(dd,(4,s[1])).reshape((-1,s[1]))
# l = dd.shape[0]
# true = np.concatenate([
#             dd[l-3-i:l-i+1] for i in range(p_len, 0, -1)
#         ], axis=1)

df_gt = pd.DataFrame(data,columns=['ds'] + [c for c in value_cols])
df_gt.head(5)

,ds,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
0,2022-01-03,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0
1,2022-01-10,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0
2,2022-01-17,910.0,12266,430.0,490.0,8791,2162,710.0,14.0,47364,20496.0
3,2022-01-24,679.0,12793,336.0,471.0,7465,2066,667.0,7.0,40234,15227.0
4,2022-01-10,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0


In [14]:
from sklearn.metrics import mean_absolute_percentage_error
SATRT = 0.95 # 4-ре недели с конца
SATRT = 0.8 # 20 недель с конца
SW = False

mapes=[]
preds=[]
data_m, data_p = [], []
for n, m in models.items():

    start = SATRT
    print(f"Model: {n}")
    for c in value_cols:
        _d = ts_dset[c]
        _p1 = m.historical_forecasts(_d, start=start, forecast_horizon=1, verbose=True,show_warnings=SW).values()
        _p2 = m.historical_forecasts(_d, start=start, forecast_horizon=2, verbose=True,show_warnings=SW).values()
        _p3 = m.historical_forecasts(_d, start=start, forecast_horizon=3, verbose=True,show_warnings=SW).values()
        _p4 = m.historical_forecasts(_d, start=start, forecast_horizon=4, verbose=True,show_warnings=SW).values()
        p_len = _p4.shape[0]
        pred = np.concatenate([
            _p1[:p_len],
            _p2[:p_len],
            _p3[:p_len],
            _p4
        ], axis=1).ravel()

        # print(f"{_p1=}, \n{_p2=}, \n{_p3=}, \n{_p4=}")
        # print(f"{_p4.shape=}, {_p1[:p_len].shape=}")
        # print(f"{true=}")
        #print(f"{pred=}")
        #preds.append(pd.Series(data=pred, name=(n, f"pr_{c}"), index=df_gt.iloc[-pred.shape[0]:]['ds']))
        preds.append(pd.Series(data=pred, name=(n, c) ))
        #print(f'{preds=}')



Model: NaiveSeasonal__{'K': 52}


  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [15]:
df_pred = pd.concat(preds, axis=1).reset_index(drop=True)
df_pred[:6]

NaiveSeasonal__{'K': 52}                                               \
               Денситометр      КТ КТ с КУ 1 зона КТ с КУ 2 и более зон   
0                   1059.0  4136.0          480.0                 476.0   
1                    896.0  4131.0          444.0                 456.0   
2                    974.0  4499.0          413.0                 506.0   
3                   1020.0  4671.0          478.0                 477.0   
4                    896.0  4131.0          444.0                 456.0   
5                    974.0  4499.0          413.0                 506.0   

                                                                               \
       ММГ     МРТ МРТ с КУ 1 зона МРТ с КУ 2 и более зон       РГ Флюорограф   
0  12199.0  1820.0           718.0                    8.0  46641.0    20380.0   
1  12446.0  1798.0           786.0                    7.0  46749.0    19369.0   
2  12262.0  1824.0           740.0                    4.0  46779.0    18481.0   
3  12730.0  1913.0           738.0                    3.0  48811.0    18390.0   
4  12446.0  1798.0           786.0                    7.0  46749.0    19369.0   
5  12262.0  1824.0           740.0                    4.0  46779.0    18481.0   

   ...  Croston__{}                                                    \
   ...  Денситометр           КТ КТ с КУ 1 зона КТ с КУ 2 и более зон   
0  ...  1570.620239  4172.245117     602.961548            702.739075   
1  ...  1570.620239  4172.245117     602.961548            702.739075   
2  ...  1570.620239  4172.245117     602.961548            702.739075   
3  ...  1570.620239  4172.245117     602.961548            702.739075   
4  ...  1561.558228  4065.220703     593.665405            691.565186   
5  ...  1561.558228  4065.220703     593.665405            691.565186   

                                                                     \
            ММГ          МРТ МРТ с КУ 1 зона МРТ с КУ 2 и более зон   
0  16399.853516  1947.107056       860.23114               9.176898   
1  16399.853516  1947.107056       860.23114               9.176898   
2  16399.853516  1947.107056       860.23114               9.176898   
3  16399.853516  1947.107056       860.23114               9.176898   
4  16414.468750  1920.196289       853.40802               9.859208   
5  16414.468750  1920.196289       853.40802               9.859208   

                               
             РГ    Флюорограф  
0  60548.125000  18609.919922  
1  60548.125000  18609.919922  
2  60548.125000  18609.919922  
3  60548.125000  18609.919922  
4  60770.410156  18462.228516  
5  60770.410156  18462.228516  

[6 rows x 70 columns]

In [16]:
true = df_gt.iloc[-len(df_pred):].reset_index(drop=True)
ds = true['ds'].to_frame()
ds.columns = pd.MultiIndex.from_product([['ds']] + [['']])

pred_mask = (ds - ds.shift(1)).astype(int)
pred_mask[(pred_mask>=0)|(pred_mask.isnull())] = 0
pred_mask[(pred_mask<0)] = 1
pred_id = pred_mask.cumsum()
pred_id.columns = pd.MultiIndex.from_product([['pred_id']] + [['']])

true = true[true.columns[1:]]
true.columns = pd.MultiIndex.from_product([['GT']] + [true.columns])

df_res = pd.concat([ds, pred_id, df_pred, true], axis=1)
df_res.head(5)

ds pred_id NaiveSeasonal__{'K': 52}                         \
                                  Денситометр      КТ КТ с КУ 1 зона   
0 2023-08-21       1                   1059.0  4136.0          480.0   
1 2023-08-28       1                    896.0  4131.0          444.0   
2 2023-09-04       1                    974.0  4499.0          413.0   
3 2023-09-11       1                   1020.0  4671.0          478.0   
4 2023-08-28       2                    896.0  4131.0          444.0   

                                                          \
  КТ с КУ 2 и более зон      ММГ     МРТ МРТ с КУ 1 зона   
0                 476.0  12199.0  1820.0           718.0   
1                 456.0  12446.0  1798.0           786.0   
2                 506.0  12262.0  1824.0           740.0   
3                 477.0  12730.0  1913.0           738.0   
4                 456.0  12446.0  1798.0           786.0   

                          ...          GT                       \
  МРТ с КУ 2 и более зон  ... Денситометр    КТ КТ с КУ 1 зона   
0                    8.0  ...      1480.0  3102          510.0   
1                    7.0  ...      1487.0  3572          495.0   
2                    4.0  ...      1481.0  3835          521.0   
3                    3.0  ...      1474.0  4038          495.0   
4                    7.0  ...      1487.0  3572          495.0   

                                                                             \
  КТ с КУ 2 и более зон    ММГ   МРТ МРТ с КУ 1 зона МРТ с КУ 2 и более зон   
0                 591.0  16546  1678           792.0                   16.0   
1                 634.0  17260  1668           752.0                   15.0   
2                 679.0  16384  1758           786.0                   11.0   
3                 670.0  17013  1680           790.0                    8.0   
4                 634.0  17260  1668           752.0                   15.0   

                     
      РГ Флюорограф  
0  62771    17133.0  
1  65026    17528.0  
2  65924    17111.0  
3  70998    16406.0  
4  65026    17528.0  

[5 rows x 82 columns]

In [17]:
df_res.to_excel(RES_DS_PATH)
# df_res.to_csv(RES_DS_PATH.with_suffix('.csv'))

In [18]:
def agg_map(df):

    ds = df[('ds','')].head(1).reset_index(drop=True)
    cols = [c for c  in df.columns if c[0] not in ("GT", "ds", "pred_id")]
    data = [mean_absolute_percentage_error(df[("GT",c[1])], df[c]) for c  in cols]
    #print(f"{data=}")
    v = pd.Series(data, index=pd.MultiIndex.from_tuples(cols)).to_frame().T
    #print(f"{v=}")
    return pd.concat([ds, v], axis=1)

metrics = df_res.groupby(('pred_id','')).apply(agg_map).reset_index(drop=True)
metrics

/tmp/ipykernel_1989801/854833868.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = df_res.groupby(('pred_id','')).apply(agg_map).reset_index(drop=True)


ds NaiveSeasonal__{'K': 52}                           \
                           Денситометр        КТ КТ с КУ 1 зона   
0  2023-08-21                 0.333061  0.204933       0.100873   
1  2023-08-28                 0.275116  0.126175       0.095266   
2  2023-09-04                 0.269600  0.094666       0.103460   
3  2023-09-11                 0.244786  0.055882       0.091745   
4  2023-09-18                 0.264619  0.038251       0.150206   
5  2023-09-25                 0.345944  0.045802       0.198828   
6  2023-10-02                 0.338121  0.059805       0.196224   
7  2023-10-09                 0.398070  0.091369       0.226086   
8  2023-10-16                 0.401951  0.083046       0.190613   
9  2023-10-23                 0.386154  0.080462       0.146328   
10 2023-10-30                 0.359946  0.077163       0.126770   
11 2023-11-06                 0.299838  0.089108       0.072184   
12 2023-11-13                 0.248675  0.120420       0.058087   
13 2023-11-20                 0.232775  0.113922       0.051070   
14 2023-11-27                 0.205115  0.120499       0.123848   
15 2023-12-04                 0.146051  0.184925       0.176691   
16 2023-12-11                 0.105608  0.275362       0.296105   
17 2023-12-18                 0.051921  0.407305       0.426577   
18 2023-12-25                 0.067398  0.450129       0.403733   
19 2024-01-01                 0.131266  0.363780       0.397888   

                                                              \
   КТ с КУ 2 и более зон       ММГ       МРТ МРТ с КУ 1 зона   
0               0.254547  0.261242  0.084699        0.065749   
1               0.279949  0.197236  0.096964        0.069949   
2               0.278980  0.209205  0.088836        0.085248   
3               0.296891  0.213853  0.090636        0.108863   
4               0.321669  0.213366  0.070191        0.129921   
5               0.306314  0.279212  0.049270        0.132733   
6               0.313228  0.263660  0.061147        0.137154   
7               0.342150  0.291288  0.085274        0.141377   
8               0.305670  0.263541  0.089250        0.108845   
9               0.298316  0.265430  0.086577        0.091208   
10              0.315306  0.271613  0.079189        0.071222   
11              0.244481  0.247932  0.046966        0.050970   
12              0.232442  0.277919  0.054111        0.063940   
13              0.229795  0.278058  0.045118        0.095176   
14              0.140494  0.267511  0.088017        0.087615   
15              0.139656  0.278798  0.101129        0.081483   
16              0.223496  0.247693  0.167157        0.157965   
17              0.306917  0.211732  0.257698        0.208430   
18              0.333981  0.208874  0.257511        0.228440   
19              0.311017  0.179652  0.265404        0.262356   

                                     ... Croston__{}                           \
   МРТ с КУ 2 и более зон        РГ  ... Денситометр        КТ КТ с КУ 1 зона   
0                0.573674  0.285237  ...    0.060882  0.158561       0.193950   
1                0.625145  0.251292  ...    0.128108  0.077111       0.168853   
2                0.541811  0.260254  ...    0.123519  0.066923       0.111986   
3                0.521609  0.251670  ...    0.121512  0.091460       0.098013   
4                0.498693  0.233569  ...    0.136322  0.131899       0.098940   
5                0.462847  0.277737  ...    0.113492  0.138992       0.118317   
6                0.559906  0.278329  ...    0.130453  0.152259       0.148386   
7                0.561642  0.308613  ...    0.148099  0.149759       0.151043   
8                0.583071  0.300278  ...    0.163968  0.134880       0.093929   
9                0.542446  0.301056  ...    0.170291  0.115377       0.069063   
10               0.484673  0.294687  ...    0.175725  0.080027       0.053572   
11               0.555586  0.279966  ...    0.173628  0.055060       0.034

In [19]:
pd.set_option('display.max_rows', None)
res = metrics.drop(columns=[('ds','')]).describe().T
res

count  \
NaiveSeasonal__{'K': 52}                      Денситометр              20.0   
                                              КТ                       20.0   
                                              КТ с КУ 1 зона           20.0   
                                              КТ с КУ 2 и более зон    20.0   
                                              ММГ                      20.0   
                                              МРТ                      20.0   
                                              МРТ с КУ 1 зона          20.0   
                                              МРТ с КУ 2 и более зон   20.0   
                                              РГ                       20.0   
                                              Флюорограф               20.0   
StatsForecastAutoTheta__{'season_length': 52} Денситометр              20.0   
                                              КТ                       20.0   
                                              КТ с КУ 1 зона           20.0   
                                              КТ с КУ 2 и более зон    20.0   
                                              ММГ                      20.0   
                                              МРТ                      20.0   
                                              МРТ с КУ 1 зона          20.0   
                                              МРТ с КУ 2 и более зон   20.0   
                                              РГ                       20.0   
                                              Флюорограф               20.0   
StatsForecastAutoARIMA__{'season_length': 52} Денситометр              20.0   
                                              КТ                       20.0   
                                              КТ с КУ 1 зона           20.0   
                                              КТ с КУ 2 и более зон    20.0   
                                              ММГ                      20.0   
                                              МРТ                      20.0   
                                              МРТ с КУ 1 зона          20.0   
                                              МРТ с КУ 2 и более зон   20.0   
                                              РГ                       20.0   
                                              Флюорограф               20.0   
StatsForecastAutoETS__{'season_length': 52}   Денситометр              20.0   
                                              КТ                       20.0   
                                              КТ с КУ 1 зона           20.0   
                                              КТ с КУ 2 и более зон    20.0   
                                              ММГ                      20.0   
                                              МРТ                      20.0   
                                              МРТ с КУ 1 зона          20.0   
                                              МРТ с КУ 2 и более зон   20.0   
                                              РГ                       20.0   
                                              Флюорограф               20.0   
StatsForecastAutoCES__{'season_length': 52}   Денситометр              20.0   
                                              КТ                       20.0   
                                              КТ с КУ 1 зона           20.0   
                                              КТ с КУ 2 и более зон    20.0   
                                              ММГ                      20.0   
                                              МРТ                      20.0   
                                              МРТ с КУ 1 зона          20.0   
                                              МРТ с КУ 2 и более зон   20.0   
                                              РГ                       20.0   
                                              Флюорограф               20.0   
KalmanForecaster__{'dim_x': 1}           

In [20]:
res.index.set_names(['model','modality'], inplace=True)
#res.reset_index()#.drop(columns='modality')
res.groupby('model').mean()

,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
Croston__{},20.0,0.189776,0.134679,0.050275,0.104140,0.147166,0.219108,0.485816
KalmanForecaster__{'dim_x': 1},20.0,0.168255,0.133943,0.042332,0.079739,0.120789,0.206156,0.501346
NaiveSeasonal__{'K': 52},20.0,0.292275,0.134754,0.137694,0.205476,0.257481,0.321805,0.586672
StatsForecastAutoARIMA__{'season_length': 52},20.0,0.159889,0.119919,0.045357,0.077418,0.121947,0.182852,0.473788
StatsForecastAutoCES__{'season_length': 52},20.0,0.163617,0.128260,0.042126,0.075046,0.118975,0.214975,0.477540
StatsForecastAutoETS__{'season_length': 52},20.0,0.157081,0.123874,0.034395,0.074905,0.116194,0.185420,0.470763
StatsForecastAutoTheta__{'season_length': 52},20.0,0.156748,0.122980,0.038936,0.073927,0.120119,0.186903,0.473068
